$ https://docs.opencv.org/4.x/d8/da4/tutorial_julia.html

[파이썬 패키지 이용](https://www.geeksforgeeks.org/how-to-import-python-packages-in-julia/)
- [오류](https://github.com/JuliaPy/PyCall.jl/issues/867)발생
```py
ENV["PYTHON"]= abspath("venv\\Scripts\\python.exe")
ENV["PYCALL_JL_RUNTIME_PYTHON"]  = abspath("venv\\Scripts\\python.exe")
ENV["PYTHONPATH"]= abspath("venv\\Lib\\site-packages")
Pkg.build("PyCall")
```
- 잘 모르겠고, 걍 전역으로 깔자.
```py
pip install opencv-python
```


In [24]:
using Pkg 
using PyCall 
@pyimport cv2
@pyimport numpy as np

In [67]:
cap = cv2.VideoCapture("data/video.MOV")
# params for ShiTomasi corner detection
feature_params = Dict( :maxCorners => 100,
                       :qualityLevel => 0.3,
                       :minDistance => 7,
                       :blockSize => 7 )
# Parameters for lucas kanade optical flow
lk_params = Dict( :winSize  => (15, 15),
                  :maxLevel => 2,
                  :criteria => (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# 확대 행렬
m_big = np.float32([[2, 0, 0], [0, 2, 0]])  

# 시작
for i = 1:30*3
    cap.read()
end

frameNm = 30*3-1

# Take first frame and find corners in it
ret, old_frame = cap.read()
# old_frame = cv2.resize(old_frame, None,  None, 2, 2, cv2.INTER_LINEAR)
# old_frame = cv2.pyrUp(old_frame)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray; feature_params...)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# 크기
cols, rows = size(old_gray)
transMat0 = nothing

# 합칠 것
output = np.zeros_like(old_frame)#.astype('float64')
reps = np.ones_like(old_frame)#.astype("float64")

hello!

1440×1920×3 Array{UInt8, 3}:
[:, :, 1] =
 0x01  0x01  0x01  0x01  0x01  0x01  …  0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01  …  0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
    ⋮                             ⋮  ⋱           ⋮                    
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01  0x01  0x01
 0x01  0x01  0x01  0x01  0x01  0x01     0x01  0x01  0x01  0x01 

In [228]:
while true
    ret, frame = cap.read()
    global frameNm += 1

    if frameNm > 7*30
        print("종료")
        break
    end

    if !ret
        print("No frames grabbed!")
        break
    end

    # frame = cv2.resize(frame, None,  None, 2, 2, cv2.INTER_LINEAR)
    # frame = cv2.pyrUp(frame)

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    global p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, nothing; lk_params...)
    # Select good points
    if p1 != nothing
        global good_new = p1[[i for (i,x) in enumerate(st) if x!=0],:,:] #p1[st==1]
        global good_old = p0[[i for (i,x) in enumerate(st) if x!=0],:,:] #p0[st==1]
    end

    # calculate 2D affine transformation matrix 
    transMat1, _ = cv2.estimateAffine2D(good_new, good_old)
    if transMat0 == nothing
        transMat0 = transMat1
    else
        # transMat0 = np.append(transMat1[:, :2]@transMat0[:, :2], (transMat1[:, 2]+transMat0[:, 2]).reshape((2,1)), axis=1)
    end
    global new_frame = cv2.warpAffine(frame, transMat0, (rows, cols))
    # print('transMat',transMat, sep='\n')

    # 더하기
    # output += new_frame # 평균
    output = maximum([new_frame;;;; output], dims=4)[:, :, :,1]
    #    max.(new_frame,output) # 최대

    reps += [(x!=0 ? 1 : 0) for x in new_frame] #new_frame.astype("bool")#.astype("float64")

    # draw the tracks
    # for (i, (new, old)) in enumerate(zip(good_new, good_old))
    #     println("new",new)
    #     a, b = new.ravel()
    #     c, d = old.ravel()
    #     mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
    #     new_frame = cv2.circle(new_frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    # end
    img = cv2.add(new_frame, mask)
    cv2.imshow("frame", cv2.pyrDown(cv2.pyrDown(img)))
    k = cv2.waitKey(30) & 0xff
    if k == 27
        break
    end
    # Now update the previous frame and previous points
    old_gray = frame_gray#.copy()
    p0 = good_new#.reshape(-1, 1, 2)
end
cv2.destroyAllWindows()

종료

In [22]:
cv2.destroyAllWindows()



In [232]:
# max
cv2.imwrite("julia_out//o1.png",output)
# cv2.imshow('outimage_max', output)
# cv2.waitKey(30)

true

In [175]:
new_frame

1440×1920×3 Array{UInt8, 3}:
[:, :, 1] =
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7  …  0xa1  0xa0  0x9f  0x9f  0xa0  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa1  0xa1  0xa0  0xa0  0xa1  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa2  0xa2  0xa1  0xa1  0xa1  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa2  0xa1  0xa1  0xa1  0xa1  0x5a
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa1  0xa1  0xa0  0xa0  0xa0  0x5a
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7  …  0xa1  0xa1  0xa0  0xa0  0xa0  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa2  0xa1  0xa1  0xa1  0xa1  0x5b
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa1  0xa1  0xa1  0xa1  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa0  0xa1  0xa1  0xa1  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa1  0xa1  0xa1  0xa1  0x5a
    ⋮                             ⋮  ⋱           ⋮                    
 0x1a  0x0d  0x06  0x04  0x0f  0x24     0x08  0x07  0x08  0x08  0x08  0x04
 0x15  0x0e  0x0a  0x06  0x00  0x06     0x07  0x05  0x06  0x07 

In [230]:
# # ntuple(i->st[i]==1,length(st))


# a = [[[5,6,2,8],[4,6,3,7]]]
# b = [[[1,2,3,4],[4,8,8,4]]]
# [[(xx,yy) for (xx,yy) in zip(x,y)] for (x,y) in zip(a,b)]

In [208]:
[1; 2]

1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×1×2 Array{Int64, 24}:
[:, :, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] =
 1

[:, :, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] =
 2

In [209]:
new_frame

1440×1920×3 Array{UInt8, 3}:
[:, :, 1] =
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7  …  0xa1  0xa0  0x9f  0x9f  0xa0  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa1  0xa1  0xa0  0xa0  0xa1  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa2  0xa2  0xa1  0xa1  0xa1  0x5b
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa2  0xa1  0xa1  0xa1  0xa1  0x5a
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7     0xa1  0xa1  0xa0  0xa0  0xa0  0x5a
 0xd7  0xd7  0xd7  0xd7  0xd7  0xd7  …  0xa1  0xa1  0xa0  0xa0  0xa0  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa2  0xa1  0xa1  0xa1  0xa1  0x5b
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa1  0xa1  0xa1  0xa1  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa0  0xa1  0xa1  0xa1  0x5a
 0xd5  0xd5  0xd5  0xd5  0xd5  0xd5     0xa1  0xa1  0xa1  0xa1  0xa1  0x5a
    ⋮                             ⋮  ⋱           ⋮                    
 0x1a  0x0d  0x06  0x04  0x0f  0x24     0x08  0x07  0x08  0x08  0x08  0x04
 0x15  0x0e  0x0a  0x06  0x00  0x06     0x07  0x05  0x06  0x07 

In [117]:
p0[[i for (i,x) in enumerate(st) if x!=0],:,:]

100×1×2 Array{Float32, 3}:
[:, :, 1] =
  342.0
  352.0
  889.0
 1153.0
 1651.0
 1715.0
  627.0
  851.0
  811.0
  331.0
    ⋮
 1163.0
 1423.0
 1716.0
 1189.0
  931.0
 1686.0
  667.0
 1777.0
  290.0

[:, :, 2] =
  910.0
  908.0
 1113.0
 1317.0
 1120.0
 1118.0
  510.0
  815.0
  734.0
  911.0
    ⋮
  944.0
  644.0
 1131.0
  631.0
  616.0
  889.0
  594.0
 1159.0
  578.0

In [98]:
p0[:,:,:]

100×1×2 Array{Float32, 3}:
[:, :, 1] =
  342.0
  352.0
  889.0
 1153.0
 1651.0
 1715.0
  627.0
  851.0
  811.0
  331.0
    ⋮
 1163.0
 1423.0
 1716.0
 1189.0
  931.0
 1686.0
  667.0
 1777.0
  290.0

[:, :, 2] =
  910.0
  908.0
 1113.0
 1317.0
 1120.0
 1118.0
  510.0
  815.0
  734.0
  911.0
    ⋮
  944.0
  644.0
 1131.0
  631.0
  616.0
  889.0
  594.0
 1159.0
  578.0